In [1]:
# Example usage of humanloop
import os
from humanloop import Humanloop
HUMANLOOP_API_KEY = os.getenv("HUMANLOOP_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

humanloop = Humanloop(api_key=HUMANLOOP_API_KEY)

response = humanloop.chat_deployed(
    project="Petkeley",
    inputs={
        "joke": "Q: Why was the broom late? A: It over swept!"
    },
    messages=[],
    provider_api_keys={
        "openai": OPENAI_API_KEY
    }
)

print(response.data[0].output)


{"Explanation": "The joke plays on the word 'swept' which relates to the broom's function of sweeping. The humor arises from the pun created by replacing 'slept', which would be a common reason for a person to be late, with 'swept', tying the reason of being late to what a broom does.", "Humor": "Pun"}


In [14]:
# read the joke from jokes.csv
import pandas as pd

jokes = pd.read_csv(r'data/jokes.csv')

# get all the jokes from column 'joke'
jokes = jokes['joke'].to_list()
print(len(jokes))


352


In [10]:
from humanloop import Humanloop
import ast

def extract_gpt_output(response):
    # change the response into a dictionary
    try:
        gpt_output = response.data[0].output
        gpt_output = ast.literal_eval(gpt_output)
        return gpt_output
    except:
        print(f'invalid response: {response.data[0].output}')
    

humanloop = Humanloop(api_key=HUMANLOOP_API_KEY)
results = [] # [(explanation, humor category), ...]

for joke in jokes:
    response = humanloop.chat_deployed(
        project="Petkeley",
        inputs={
            "joke": f'{joke}'
        },
        messages=[],
        provider_api_keys={
            "openai": OPENAI_API_KEY
        }
    )
    gpt_output = extract_gpt_output(response)
    
    if gpt_output:
        results.append((gpt_output['Explanation'], gpt_output['Humor']))
    
    

In [11]:
print(len(results))

352


In [12]:
# save the results to a csv file
results_df = pd.DataFrame(results, columns=['Explanation', 'Humor'])
results_df.to_csv('data/gpt4turbo_explained_full.csv', index=False)

In [13]:
# show the number of each different kind of humor
results_df['Humor'].value_counts()


Humor
Pun           332
Wordplay       10
Absurd          4
Irony           2
Analogy         1
Twist           1
Literal         1
Anti-humor      1
Name: count, dtype: int64